In [240]:
import numpy as np

f = open("input.dat")
lines = f.readlines()
lines = [line.strip() for line in lines]
f.close()

idx_breaks = [idx for idx, line in enumerate(lines) if line==""]
idx_breaks
patterns = [[] for _ in range(len(idx_breaks)+1)]
patterns_v = [[] for _ in range(len(idx_breaks)+1)]
start_parse = 0

for i, idx in enumerate(idx_breaks):
    patterns[i] = np.array([[c for c in line] for line in lines[start_parse:idx]])
    start_parse = idx+1
patterns[-1] = np.array([[c for c in line] for line in lines[idx+1:]])

for ip, (pat,pat_v) in enumerate(zip(patterns, patterns_v)):
    patterns_v[ip] = pat.transpose()


def search_match(pat):
    idx_m = []
    for i, p in enumerate(pat[:-1]):
        if np.all(p == pat[i+1]):
            print("Match!", p)
            idx_m.append(i)

    return idx_m

def validate_matches(match_id:list, pat:np.ndarray):
    print('validate:', match_id)
    match_id_pairs = []
    for i, id in enumerate(match_id):
        idx_left = [i for i in reversed(range(id))]
        idx_right = [i for i in range(id+2,len(pat))]
        print('scanning...', id, idx_left, idx_right)
        has_pair=True
        for il, ir in zip(idx_left,idx_right):
            if np.any(pat[il] != pat[ir]):
                print(pat[il], pat[ir])
                has_pair=False
                break
        if has_pair:
            match_id_pairs.append(id)
    
    if len(match_id_pairs) > 1:
        assert False, print(match_id_pairs)
    print('done validate', match_id_pairs)

    if len(match_id_pairs)  == 1:
        nlines = len(pat)
        nup = (match_id_pairs[0]+1) #idx at 0
        # ndown = nlines - (match_id_pairs[0]+1)
        # if ndown==1:  #canse where the last two are a match
        value = nup
        # else:
        #     value = np.min([nup,ndown]) #from the match
        # print(value, nlines, nup, ndown)
        return value
    else:
        return 0
    

ret = 0
for ip, (pat,pat_v) in enumerate(zip(patterns, patterns_v)):
    match_id = search_match(pat)
    print('Horizintal')
    print('Matches start:',match_id)
    for p in pat:
        for pp in p:
            print(pp, end='')
        print()
    match_id = validate_matches(match_id, pat)

    match_id_v = search_match(pat_v)
    print('Vertical')
    print('Matches start:',match_id_v)
    for p in pat_v:
        print(p)
    match_id_v = validate_matches(match_id_v, pat_v)
    print()
    if match_id==0 and match_id_v==0:
        assert False, print("They match?", match_id, match_id_v)
    ret+=match_id*100
    ret+=match_id_v
    print()
print(ret)

Match! ['.' '.' '.' '#' '#' '#' '.' '#' '#' '.' '#' '.' '#' '.' '.']
Match! ['.' '.' '.' '.' '.' '#' '#' '#' '.' '#' '.' '#' '.' '.' '#']
Horizintal
Matches start: [1, 9]
.####.#...##.#.
...###.##.#.#..
...###.##.#.#..
.####.#...#..#.
..##......#..#.
#.#...#.##...##
.#.#.#..##..##.
#..##...#####.#
.#.#.#.#..##.#.
.....###.#.#..#
.....###.#.#..#
validate: [1, 9]
scanning... 1 [0] [3, 4, 5, 6, 7, 8, 9, 10]
['.' '#' '#' '#' '#' '.' '#' '.' '.' '.' '#' '#' '.' '#' '.'] ['.' '#' '#' '#' '#' '.' '#' '.' '.' '.' '#' '.' '.' '#' '.']
scanning... 9 [8, 7, 6, 5, 4, 3, 2, 1, 0] []
done validate [9]
Vertical
Matches start: []
['.' '.' '.' '.' '.' '#' '.' '#' '.' '.' '.']
['#' '.' '.' '#' '.' '.' '#' '.' '#' '.' '.']
['#' '.' '.' '#' '#' '#' '.' '.' '.' '.' '.']
['#' '#' '#' '#' '#' '.' '#' '#' '#' '.' '.']
['#' '#' '#' '#' '.' '.' '.' '#' '.' '.' '.']
['.' '#' '#' '.' '.' '.' '#' '.' '#' '#' '#']
['#' '.' '.' '#' '.' '#' '.' '.' '.' '#' '#']
['.' '#' '#' '.' '.' '.' '.' '.' '#' '#' '#']
['.' '#' '

In [267]:

import numpy as np

f = open("input.dat")
lines = f.readlines()
lines = [line.strip() for line in lines]
f.close()

idx_breaks = [idx for idx, line in enumerate(lines) if line==""]
idx_breaks
patterns = [[] for _ in range(len(idx_breaks)+1)]
patterns_v = [[] for _ in range(len(idx_breaks)+1)]
start_parse = 0

for i, idx in enumerate(idx_breaks):
    patterns[i] = np.array([[c for c in line] for line in lines[start_parse:idx]])
    start_parse = idx+1
patterns[-1] = np.array([[c for c in line] for line in lines[idx+1:]])

for ip, (pat,pat_v) in enumerate(zip(patterns, patterns_v)):
    patterns_v[ip] = pat.transpose()


def test_smudged(pat:np.ndarray, il:int, ir:int):
    matches = ~np.array(pat[il] == pat[ir])
    if np.count_nonzero(matches)==1:
        return True
        # idx_smudge = matches.argmin()
        # I could change the array here, but I don't think it is necessary...
    else:
        return False
def check_reflections(pat:np.ndarray, idx_mirror:int):
    idx_left = [i for i in reversed(range(idx_mirror))]
    idx_right = [i for i in range((idx_mirror+1)+1,len(pat))]
    # print('scanning...', idix_mirror,_left, idx_right)
    has_pair=True
    smudged=False
    for il, ir in zip(idx_left,idx_right):
        if np.any(pat[il] != pat[ir]):
            has_pair=False
            if test_smudged(pat, il, ir):
                smudged = True
                has_pair = True
            else:
                break
    return has_pair, smudged


def validate_matches(pat:np.ndarray):
    
    for i, p in enumerate(pat[:-1]): # scanning every possible mirror point
        smudged = False
        smudged = test_smudged(pat,i, i+1)
        if np.all(p == pat[i+1]) or smudged:
            valid, smudged2 = check_reflections(pat, i)
            if valid and (smudged or smudged2):
                nup = (i+1) #idx at 0
                value = nup
                return value
    
    return 0


ret = 0
for ip, (pat,pat_v) in enumerate(zip(patterns, patterns_v)):
    print('Horizintal')
    for p in pat:
        for pp in p:
            print(pp, end='')
        print()
    print()
    match_id = validate_matches(pat)

    match_id_v = 0
    if match_id==0:
        print('Vertical')
        for p in pat_v:
            for pp in p:
                print(pp, end='')
            print()
        print()
        match_id_v = validate_matches(pat_v)

    ret+=match_id*100
    ret+=match_id_v
print(ret)

Horizintal
.####.#...##.#.
...###.##.#.#..
...###.##.#.#..
.####.#...#..#.
..##......#..#.
#.#...#.##...##
.#.#.#..##..##.
#..##...#####.#
.#.#.#.#..##.#.
.....###.#.#..#
.....###.#.#..#

Horizintal
.#.###.....
#..#.##...#
.#.....#.#.
.#.....#.#.
#..#.##...#
.#.###.....
##.#..##.##
#....##..#.
.##.##.#.##
...#..###.#
...#..###.#
.##.##.#.##
#....##..#.
####..##.##
.#.###.....
#..#.##...#
.#.....#.#.

Horizintal
###....##....
####..#..#..#
..###.####.##
..#..######..
...#.######.#
#####.#.##.##
##.#..#..#..#
##....####...
....#......#.
.....#.##.#..
##...##..##..
...##.#..#.##
##..###..###.

Vertical
##...###..#.#
##...###..#.#
####.#.......
.##.###....#.
..#..#..#..##
...##....##.#
.#######..###
#.###..#.#...
#.####.#.#...
.#######..###
...##....##.#
..#..#..#..##
.##.###....#.

Horizintal
.##.#...##.
.#.########
.##.#.#.##.
#.##..#....
####.##....
####.##....
#.##..#....
.##.#.#.##.
.#.########
.##.#...##.
..####.####
.##.###.###
.#..#.#.##.
#.####..##.
...#..#....

Vertical
...####..

In [199]:
a = [1,2,2,2]

for i,k in enumerate(a):
    print(k)
    if k!=1:
        a.pop(i)
a

1
2
2


[1, 2]

/home/henrique/Dropbox/APC_Paris/Root/adventofcode2023/13
